#VI-EN Machine Translation using BERT-to-BERT Model
**Dataset: IWSLT15-en-vi**

**khuongvd00@gmail.com**

In [ ]:
!nvidia-smi

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/NLP/machine_translation

In [3]:
!pip install -q transformers==4.26.1 datasets==2.9.0 sacrebleu==2.3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.4 MB/s eta 0:00:00


In [13]:
import os
import numpy as np

import sacrebleu

import torch
from torch.utils.data import Dataset

from datasets import load_dataset, load_metric
from transformers import *

##1.Prepare Data

In [5]:
class NMTDataset(Dataset):
    def __init__(self, cfg, data_type="train"):
        super().__init__()
        self.cfg = cfg

        self.src_texts, self.tgt_texts = self.read_data(data_type)

        self.src_input_ids, self.src_attention_mask = self.texts_to_sequences(self.src_texts)
        self.tgt_input_ids, self.tgt_attention_mask, self.labels = self.texts_to_sequences(
            self.tgt_texts, 
            is_src=False
        )

    def read_data(self, data_type):
        data = load_dataset(
            "mt_eng_vietnamese", 
            "iwslt2015-en-vi", 
            split=data_type
        )
        src_texts = [sample["translation"][self.cfg.src_lang] for sample in data]
        tgt_texts = [sample["translation"][self.cfg.tgt_lang] for sample in data]
        return src_texts, tgt_texts
    
    def texts_to_sequences(self, texts, is_src=True):
        if is_src:
            src_inputs = self.cfg.src_tokenizer(
                texts,
                padding='max_length',
                truncation=True,
                max_length=self.cfg.src_max_len,
                return_tensors='pt'
            )
            return (
                src_inputs.input_ids, 
                src_inputs.attention_mask
            )

        else:
            if self.cfg.add_special_tokens:
                texts = [
                    ' '.join([
                        self.cfg.tgt_tokenizer.bos_token,
                        text,
                        self.cfg.tgt_tokenizer.eos_token
                        ]) 
                    for text in texts
                ]
            tgt_inputs = self.cfg.tgt_tokenizer(
                texts,
                padding='max_length',
                truncation=True,
                max_length=self.cfg.tgt_max_len,
                return_tensors='pt'
            )

            labels = tgt_inputs.input_ids.numpy().tolist()
            labels = [
                [
                    -100 if token_id == self.cfg.tgt_tokenizer.pad_token_id else token_id
                    for token_id in label
                ]
                for label in labels
            ]

            labels = torch.LongTensor(labels)

            return (
                tgt_inputs.input_ids,
                tgt_inputs.attention_mask,
                labels
            )

    def __getitem__(self, idx):
        return {
            "input_ids": self.src_input_ids[idx],
            "attention_mask": self.src_attention_mask[idx],
            "decoder_input_ids": self.tgt_input_ids[idx],
            "decoder_attention_mask": self.tgt_attention_mask[idx],
            "labels": self.labels[idx]
        }

    def __len__(self):
        return np.shape(self.src_input_ids)[0]

##2.Load Tokenizer and Model

In [6]:
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels

def load_tokenizer(model_name_or_path):
    if 'bert' in model_name_or_path.split('-'):
        return BertTokenizerFast.from_pretrained(model_name_or_path)
    elif 'gpt2' in model_name_or_path.split('-'):
        return GPT2TokenizerFast.from_pretrained(model_name_or_path)
    else:
        return AutoTokenizer.from_pretrained(model_name_or_path)

In [ ]:
class Manager():
    def __init__(self, cfg, is_train=True):
        self.cfg = cfg

        print("Loading Tokenizer...")
        self.get_tokenizer()

        print("Loading Model...")
        self.get_model()

        print("Loading Metric...")
        self.bleu_metric = load_metric("sacrebleu")

        print("Check Save Model Path")
        if not os.path.exists(self.cfg.ckpt_dir):
            os.mkdir(self.cfg.ckpt_dir)

        if is_train:
            # Load dataloaders
            print("Loading Dataset...")
            self.train_dataset = NMTDataset(self.cfg, data_type="train")
            self.valid_dataset = NMTDataset(self.cfg, data_type="validation")
        
        print("Setting finished.")

    def get_tokenizer(self):
        if self.cfg.load_model_from_path:
            self.cfg.src_tokenizer = load_tokenizer(self.cfg.ckpt_dir)
            self.cfg.tgt_tokenizer = load_tokenizer(self.cfg.ckpt_dir)
        else:
            self.cfg.src_tokenizer = load_tokenizer(self.cfg.src_model_name)
            self.cfg.tgt_tokenizer = load_tokenizer(self.cfg.tgt_model_name)
            if "bert" in self.cfg.tgt_model_name.split('-'):
                self.cfg.add_special_tokens = False
                self.cfg.bos_token_id = self.cfg.tgt_tokenizer.cls_token_id
                self.cfg.eos_token_id = self.cfg.tgt_tokenizer.sep_token_id
                self.cfg.pad_token_id = self.cfg.tgt_tokenizer.pad_token_id
            else:
                self.cfg.add_special_tokens = True
                self.cfg.tgt_tokenizer.add_special_tokens(
                    {
                        "bos_token": "[BOS]", 
                        "eos_token": "[EOS]", 
                        "pad_token": "[PAD]"
                    }
                )
                self.cfg.bos_token_id = self.cfg.tgt_tokenizer.bos_token_id
                self.cfg.eos_token_id = self.cfg.tgt_tokenizer.eos_token_id
                self.cfg.pad_token_id = self.cfg.tgt_tokenizer.pad_token_id
        self.cfg.src_tokenizer.save_pretrained(
                os.path.join(self.cfg.ckpt_dir, f"{self.cfg.src_lang}_tokenizer_{cfg.src_model_name}")
            )
        
        self.cfg.tgt_tokenizer.save_pretrained(
                os.path.join(self.cfg.ckpt_dir, f"{self.cfg.tgt_lang}_tokenizer_{cfg.tgt_model_name}")
            )
    
    def get_model(self):
        if self.cfg.load_model_from_path:
            save_model_path = os.path.join(self.cfg.ckpt_dir, self.cfg.ckpt_name)
            self.model = EncoderDecoderModel.from_pretrained(save_model_path)
        else:
            self.model = EncoderDecoderModel.from_encoder_decoder_pretrained(
                self.cfg.src_model_name, 
                self.cfg.tgt_model_name
            )
            self.model.decoder.resize_token_embeddings(len(self.cfg.tgt_tokenizer))
            self.model.config.decoder_start_token_id = self.cfg.bos_token_id
            self.model.config.eos_token_id = self.cfg.eos_token_id
            self.model.config.pad_token_id = self.cfg.pad_token_id
            self.model.config.vocab_size = len(self.cfg.tgt_tokenizer)
            self.model.config.max_length = self.cfg.max_length_decoder
            self.model.config.min_length = self.cfg.min_length_decoder
            self.model.config.no_repeat_ngram_size = 3
            self.model.config.early_stopping = True
            self.model.config.length_penalty = 2.0
            self.model.config.num_beams = self.cfg.beam_size

    def train(self):
        print("Training...")
        if self.cfg.use_eval_steps:
            training_args = Seq2SeqTrainingArguments(
                predict_with_generate=True,
                evaluation_strategy="steps",
                save_strategy='steps',
                save_steps=self.cfg.eval_steps,
                eval_steps=self.cfg.eval_steps,
                output_dir=self.cfg.ckpt_dir,
                per_device_train_batch_size=self.cfg.train_batch_size,
                per_device_eval_batch_size=self.cfg.eval_batch_size,
                learning_rate=self.cfg.learning_rate,
                weight_decay=0.005,
                num_train_epochs=self.cfg.num_train_epochs
            )
        else:
            training_args = Seq2SeqTrainingArguments(
                predict_with_generate=True,
                evaluation_strategy="epoch",
                save_strategy='epoch',
                output_dir=self.cfg.ckpt_dir,
                per_device_train_batch_size=self.cfg.train_batch_size,
                per_device_eval_batch_size=self.cfg.eval_batch_size,
                learning_rate=self.cfg.learning_rate,
                weight_decay=0.005,
                num_train_epochs=self.cfg.num_train_epochs
            )

        data_collator = DataCollatorForSeq2Seq(
            self.cfg.tgt_tokenizer, 
            model=self.model
        )

        trainer = Seq2SeqTrainer(
            self.model,
            training_args,
            train_dataset=self.train_dataset,
            eval_dataset=self.valid_dataset,
            data_collator=data_collator,
            tokenizer=self.cfg.tgt_tokenizer,
            compute_metrics=self.compute_metrics
        )

        trainer.train()

    def compute_metrics(self, eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]
        decoded_preds = self.cfg.tgt_tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = np.where(labels != -100, labels, self.cfg.tgt_tokenizer.pad_token_id)
        decoded_labels = self.cfg.tgt_tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

        result = self.bleu_metric.compute(
            predictions=decoded_preds, 
            references=decoded_labels
        )

        result = {"bleu_score": result["score"]}

        prediction_lens = [np.count_nonzero(pred != self.cfg.tgt_tokenizer.pad_token_id) for pred in preds]
        result["gen_len"] = np.mean(prediction_lens)
        result = {k: round(v, 4) for k, v in result.items()}

        return result

##3.Config

In [ ]:
class BaseConfig:
    """ base Encoder Decoder config """

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class NMTConfig(BaseConfig):
    # Data
    src_lang = 'vi'
    tgt_lang = 'en'
    src_max_len = 75
    tgt_max_len = 75

    # Model
    src_model_name = "bert-base-multilingual-cased"
    tgt_model_name = "bert-base-multilingual-cased"

    # Training
    load_model_from_path = False
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    learning_rate = 3e-5
    train_batch_size = 16
    eval_batch_size = 8
    num_train_epochs =15
    ckpt_dir = src_model_name + '_to_' + tgt_model_name
    use_eval_steps = False
    eval_steps = 2000

    # Inference
    max_length_decoder = 75
    min_length_decoder = 25
    beam_size = 5

cfg = NMTConfig()

##4.Training

In [ ]:
manager = Manager(cfg, is_train=True)

Loading Tokenizer...


loading file vocab.txt from cache at /home/aivn2023/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/vocab.txt
loading file tokenizer.json from cache at /home/aivn2023/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /home/aivn2023/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/tokenizer_config.json
loading configuration file config.json from cache at /home/aivn2023/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropou

Loading Model...


loading configuration file config.json from cache at /home/aivn2023/.cache/huggingface/hub/models--bert-base-multilingual-cased/snapshots/fdfce55e83dbed325647a63e7e1f5de19f0382ba/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-multilingual-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "voc

Generation config file not found, using a generation config created from the model config.
Set `config.is_decoder=True` and `config.add_cross_attention=True` for decoder_config
Generate config GenerationConfig {
  "transformers_version": "4.26.1"
}

/tmp/ipykernel_430363/4108036223.py:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  self.bleu_metric = load_metric("sacrebleu")


Loading Metric...
Check Save Model Path
Loading Dataset...


Found cached dataset mt_eng_vietnamese (/home/aivn2023/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-en-vi/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71)
Found cached dataset mt_eng_vietnamese (/home/aivn2023/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-en-vi/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71)


Setting finished.


In [ ]:
manager.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Training...


/home/aivn2023/anaconda3/envs/thainq_env/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 133318
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 124995
  Number of trainable parameters = 384194811
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/aivn2023/anaconda3/envs/thainq_env/lib/python3.8/site-packages/transformers/models/encoder_decoder/modeling_encoder_decoder.py:63

Epoch,Training Loss,Validation Loss,Bleu Score,Gen Len
1,2.043300,1.902137,19.662200,35.817200
2,1.701000,1.706498,23.132900,34.943300
3,1.500400,1.618314,24.390700,34.435800
4,1.343300,1.598324,24.920100,34.556300
5,1.216100,1.617466,24.958200,34.854200
6,1.087700,1.674030,25.264200,34.464100
7,0.982900,1.627817,25.067000,33.966100
8,0.878500,1.690365,25.077800,34.735200
9,0.818400,1.783687,24.937000,34.779400
10,0.724200,1.772490,24.786100,34.627300


***** Running Evaluation *****
  Num examples = 1269
  Batch size = 8
Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

/home/aivn2023/anaconda3/envs/thainq_env/lib/python3.8/site-packages/transformers/generation/utils.py:1186: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,

...

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty":

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty":

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty":

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty":

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty": 2.0,
  "max_length": 75,
  "min_length": 25,
  "no_repeat_ngram_size": 3,
  "num_beams": 5,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 101,
  "early_stopping": true,
  "eos_token_id": 102,
  "length_penalty":

##5.Evaluate

In [5]:
def load_model(cfg, checkpoint_name):
    # Load Tokenizer
    src_tokenizer_save_path = f"{cfg.ckpt_dir}/{cfg.src_lang}_tokenizer_{cfg.src_model_name}"
    src_tokenizer = BertTokenizerFast.from_pretrained(src_tokenizer_save_path)

    tgt_tokenizer_save_path = f"{cfg.ckpt_dir}/{cfg.tgt_lang}_tokenizer_{cfg.tgt_model_name}"
    tgt_tokenizer = GPT2TokenizerFast.from_pretrained(tgt_tokenizer_save_path)

    # Load Model
    model_save_path = f"{cfg.ckpt_dir}/{checkpoint_name}"
    model = EncoderDecoderModel.from_pretrained(model_save_path)

    # Inference Param
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

    return src_tokenizer, tgt_tokenizer, model, device

In [10]:
from tqdm import tqdm
def inference(
    text, 
    src_tokenizer, 
    tgt_tokenizer, 
    model, 
    device="cpu", 
    max_length=75,
    beam_size=5
    ):
    inputs = src_tokenizer(
        text, 
        padding="max_length", 
        truncation=True, 
        max_length=max_length, 
        return_tensors="pt"
        )
    input_ids = inputs.input_ids.to(device)
    attention_mask = inputs.attention_mask.to(device)
    model.to(device)

    outputs = model.generate(
        input_ids, 
        attention_mask=attention_mask, 
        max_length=max_length, 
        early_stopping=True, 
        num_beams=beam_size, 
        length_penalty=2.0
    )

    output_str = tgt_tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return output_str

def inference_bath(
    texts, 
    src_tokenizer, 
    tgt_tokenizer, 
    model, 
    device="cpu", 
    max_length=75,
    beam_size=5,
    batch_size=32
    ):

    pred_texts = []

    if len(texts) < batch_size:
        batch_size = len(texts)

    for x in tqdm(range(0, len(texts), batch_size)):
        text = texts[x:x+batch_size]

        inputs = src_tokenizer(
            text, 
            padding="max_length", 
            truncation=True, 
            max_length=max_length, 
            return_tensors="pt"
            )
        
        input_ids = inputs.input_ids.to(device)
        attention_mask = inputs.attention_mask.to(device)
        model.to(device)

        outputs = model.generate(
            input_ids, 
            attention_mask=attention_mask, 
            max_length=max_length, 
            early_stopping=True, 
            num_beams=beam_size, 
            length_penalty=2.0
        )

        output_str = tgt_tokenizer.batch_decode(outputs, skip_special_tokens=True)
        pred_texts.extend(output_str)
        torch.cuda.empty_cache()

    return pred_texts

In [7]:
class BaseConfig:
    """ base Encoder Decoder config """

    def __init__(self, **kwargs):
        for k, v in kwargs.items():
            setattr(self, k, v)

class NMTConfig(BaseConfig):
    # Data
    src_lang = 'vi'
    tgt_lang = 'en'
    src_max_len = 75
    tgt_max_len = 75

    # Model
    src_model_name = "bert-base-multilingual-cased"
    tgt_model_name = "bert-base-multilingual-cased"

    # Training
    load_model_from_path = False
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    learning_rate = 3e-5
    train_batch_size = 16
    eval_batch_size = 8
    num_train_epochs =15
    ckpt_dir = src_model_name + '_to_' + tgt_model_name
    use_eval_steps = False
    eval_steps = 2000

    # Inference
    max_length_decoder = 75
    min_length_decoder = 25
    beam_size = 5

cfg = NMTConfig()

In [8]:
data = load_dataset("mt_eng_vietnamese", "iwslt2015-en-vi", split="test")
src_texts = [sample["translation"]["vi"] for sample in data]
tgt_texts = [sample["translation"]["en"] for sample in data]

Generating train split:   0%|          | 0/133318 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1269 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1269 [00:00<?, ? examples/s]

Dataset mt_eng_vietnamese downloaded and prepared to /root/.cache/huggingface/datasets/mt_eng_vietnamese/iwslt2015-en-vi/1.0.0/53add551a01e9874588066f89d42925f9fad43db347199dad00f7e4b0c905a71. Subsequent calls will reuse this data.


###Checkpoint-49998 (Best Valid Bleu Score)

In [ ]:
src_tokenizer, tgt_tokenizer, model, device = load_model(cfg, checkpoint_name="checkpoint-49998")

**beam_size=1**

In [ ]:
pred_texts = inference_bath(src_texts, src_tokenizer, tgt_tokenizer, model, device, beam_size=1)

In [23]:
sacrebleu.corpus_bleu(pred_texts, [tgt_texts])

BLEU = 23.06 48.2/28.0/17.9/11.7 (BP = 1.000 ratio = 1.246 hyp_len = 35245 ref_len = 28297)

**beam_size=5**

In [ ]:
pred_texts = inference_bath(src_texts, src_tokenizer, tgt_tokenizer, model, device)

In [19]:
sacrebleu.corpus_bleu(pred_texts, [tgt_texts])

BLEU = 23.72 49.5/28.7/18.4/12.1 (BP = 1.000 ratio = 1.231 hyp_len = 34841 ref_len = 28297)

###Checkpoint-33332 (Best Valid Loss)

In [ ]:
src_tokenizer, tgt_tokenizer, model, device = load_model(cfg, checkpoint_name="checkpoint-33332")

**beam_size=1**

In [ ]:
pred_texts = inference_bath(src_texts, src_tokenizer, tgt_tokenizer, model, device, beam_size=1)

In [26]:
sacrebleu.corpus_bleu(pred_texts, [tgt_texts])

BLEU = 21.66 46.5/26.6/16.7/10.6 (BP = 1.000 ratio = 1.284 hyp_len = 36344 ref_len = 28297)

**beam_size=5**

In [ ]:
pred_texts = inference_bath(src_texts, src_tokenizer, tgt_tokenizer, model, device)

In [28]:
sacrebleu.corpus_bleu(pred_texts, [tgt_texts])

BLEU = 23.38 49.1/28.4/18.2/11.8 (BP = 1.000 ratio = 1.243 hyp_len = 35182 ref_len = 28297)